<a href="https://colab.research.google.com/github/menpente/somosnlp_reescritura_textos_administrativos/blob/main/Prueba_sin_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import pandas as pd


In [ ]:
def scrape_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove script, style, meta, link, noscript, header, footer, aside, nav, img tags
        for tag in soup(["script", "style", "meta", "link", "noscript", "header", "footer", "aside", "nav", "img"]):
            tag.extract()

        # Elimina el contenido del mensaje de cookies
        for cookie_alert in soup.find_all("div", class_=["alertaModal", "msg cookies oculto"]):
            cookie_alert.decompose()

        # Extraer el texto manteniendo el orden de párrafos y listas
        visible_texts = []
        for element in soup.body.find_all(['p', 'li']):  # Busca párrafos y elementos de lista directamente bajo <body>
            text = element.get_text(separator=' ', strip=True)
            if text:  # Asegúrate de que el texto no esté vacío
                visible_texts.append(text)

        # Combina todos los textos extraídos en una única cadena, separando los elementos con un espacio
        visible_text = " ".join(visible_texts)

        # Limpiar el texto de espacios múltiples y saltos de línea
        visible_text = re.sub(r'\s+', ' ', visible_text)

        return visible_text.strip()

    except Exception as e:
        print(f"Error occurred while scraping the data: {e}")
        return None


In [ ]:
#Extraer links:



def extraer_links(link):
# URL base del sitio del que se extraerán los enlaces
  base_url = link

  response = requests.get(base_url)  # Haciendo la petición a la página principal
  response.raise_for_status()
  soup = BeautifulSoup(response.content, 'html.parser')

  links = set()  # Utiliza un conjunto para evitar duplicados
  for link in soup.find_all('a'):
      href = link.get('href')
      if href and not href.startswith('#'):  # Ignorar enlaces internos de la página que comienzan con '#'
        full_url = urljoin(base_url, href)  # Convierte rutas relativas en absolutas usando la URL base
        links.add(full_url)
  df_enlaces = pd.DataFrame(list(links), columns=['Enlaces'])

  return df_enlaces

In [ ]:
df_enlaces = extraer_links("https://www.comunidad.madrid/")


df_enlaces['texto'] = df_enlaces['Enlaces'].apply(scrape_text)

Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'
Error occurred while scraping the data: 400 Client Error: Bad Request for url: https://twitter.com/ComunidadMadrid
Error occurred while scraping the data: 404 Client Error: Not Found for url: https://www.comunidad.madrid/transparencia/plan-de-gobierno/presidente
Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


In [ ]:
!pip install groq

from groq import Groq
from google.colab import userdata

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
def eval_text (text):
  prompt = """Eres un experto en lenguaje claro en español.
  Dado un texto "sucio" extraído de una página web, realizarás las siguientes acciones:
1. Limpiar el texto, conservando solo la información relevante y eliminando palabras que correspondan a redes sociales, etiquetas de menús, etc.
2. Leer y comprender el texto, extrayendo la información que consideres útil para el usuario.
3. Empezarás directamente con el texto aclarado, sin ninguna aclaración o instrucción antes.
4. Escribirás listas si se detalla algún procedimiento.
5. Utilizarás un lenguaje sencillo que todo el mundo pueda entender. Si encuentras alguna palabra difícil, sustitúyela por otra palabra o expresión que sea más fácil de entender.
6. Reescribirás el texto basado en tu comprensión del mismo. Al hacerlo, utilizarás solo oraciones en voz activa que preserven el orden natural de las oraciones, eliminarás términos técnicos y palabras que sean difíciles de entender.
ADVERTENCIA: ¡RECUERDA SIEMPRE QUE SOLO PUEDES RESPONDER EN ESPAÑOL para que el usuario pueda entenderte! De lo contrario, el texto no servirá de nada.
El texto es el siguiente
"""
  if text is None:
    return " "

  if text is not None:
    input = prompt + text
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input,
                }
    ],
        model="mixtral-8x7b-32768"
        )
    corrected_text = (chat_completion.choices[0].message.content)
  return corrected_text


In [ ]:
df_enlaces.loc[:10, 'texto corregido'] = df_enlaces['texto'].iloc[:11].apply(eval_text)

In [ ]:
df_enlaces.to_csv('datset.csv')

##TERRITORIO DE PRUEBAS

In [ ]:

url_getafe = "https://sede.getafe.es/sta/CarpetaPublic/doEvent?APP_CODE=STA&PAGE_CODE=CATALOGO&DETALLE=6269000961339052507583"
texto_scrap_getafe = scrape_text(url_getafe)

pprint.pprint(texto_scrap_getafe)



('El volante/certificado de empadronamiento es un documento de carácter '
 'informativo en relación a la acreditación de residencia y domicilio habitual '
 'en el municipio, a efectos administrativos. El volante de empadronamiento y '
 'el certificado de empadronamiento pueden ser: Individual: donde sólo constan '
 'los datos del solicitante. Familiar o Colectivo: donde constan todas las '
 'personas empadronadas en ese domicilio. Histórico Individual o Colectivo: '
 'donde consta de forma individual o colectiva el tiempo empadronado en el '
 'domicilio. LOS VOLANTES/CERTIFICADOS INDIVIDUALES puede solicitarse, además '
 'de presencialmente, online del siguiente modo: Con certificado electrónico: '
 'a través de la opción de PADRON DE HABITANTES de " Mi Carpeta Ciudadana " y '
 'podrá descargarlo inmediatamente. Sin certificado electrónico: a través de '
 'la Oficina Virtual puede solicitar volante/certificado individual que le '
 'será remitido por correo ordinario a la dirección en l

In [ ]:
texto_getafe_aclarado = eval_text(texto_scrap_getafe)
pprint.pprint(texto_getafe_aclarado)

('El volante/certificado de empadronamiento es un documento informativo sobre '
 'tu residencia y domicilio principal en el municipio para fines '
 'administrativos. Puede ser individual, con solo tus datos, o '
 'familiar/colectivo, con datos de todas las personas empadronadas en esa '
 'dirección. También puede ser histórico, mostrando el tiempo que has vivido '
 'en esa dirección.\n'
 '\n'
 'Puedes solicitar el volante/certificado individual:\n'
 '\n'
 '* Presencialmente\n'
 '* En línea con certificado electrónico: ve a "Mi Carpeta Ciudadana" y '
 'descárgalo inmediatamente.\n'
 '* En línea sin certificado electrónico: solicítalo a través de la Oficina '
 'Virtual y te lo enviarán por correo ordinario a tu dirección de residencia '
 'registrada.\n'
 '\n'
 'Para solicitar en línea sin certificado electrónico:\n'
 '\n'
 '* Usa un ordenador, no un dispositivo móvil.\n'
 '* Accede con los siguientes navegadores: Internet Explorer 7.0 o superior, '
 'Mozilla Firefox 3.5 o superior, Googl